In [24]:
from pathlib import Path
from langchain.agents import initialize_agent, Tool
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine
import sqlite3
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from table_relationships import describe_table_relationships
from tbl_col_info import table_info_and_examples
import os
import pandas as pd
import re
import requests

In [25]:
# Load .env file and get GROQ API key
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [26]:
if not api_key:
    print("GROQ_API_KEY not found in .env file.")

In [27]:
# Setup the LLM
llm = ChatGroq(groq_api_key=api_key, model_name="Llama3-8b-8192", streaming=True)

In [28]:
# Configure and return SQLite database connection
def configure_db():
    import glob
    conn = sqlite3.connect(":memory:")
    csv_folder = Path.cwd() / "cooked_data_gk"
    for csv_file in glob.glob(str(csv_folder / "*.csv")):
        table_name = Path(csv_file).stem.lower()
        df = pd.read_csv(csv_file)
        df.to_sql(table_name, conn, index=False, if_exists="replace")
    return SQLDatabase.from_uri("sqlite://", engine_args={"creator": lambda: conn})

# Connect to DB
db = configure_db()
print("📄 Tables Loaded:", db.get_table_names())

📄 Tables Loaded: ['order_items', 'orders', 'sub_d_order_items', 'sub_d_orders', 'tbl_distributor_details', 'tbl_price_margin', 'tbl_product_master', 'tbl_sales', 'tbl_scheme']


In [31]:
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['order_items', 'orders', 'sub_d_order_items', 'sub_d_orders', 'tbl_distributor_details', 'tbl_price_margin', 'tbl_product_master', 'tbl_sales', 'tbl_scheme']


In [21]:
# Tools
relationship_tool = Tool(
    name="TableRelationships",
    func=describe_table_relationships,
    description="Use this tool to understand how tables are related before writing SQL queries."
)
info_example_tool = Tool(
    name="TableInfoAndExamples",
    func=table_info_and_examples,
    description="Use this tool to understand available tables and columns see example queries.",
)

In [22]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools() + [relationship_tool, info_example_tool]

In [23]:
tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FA8A025950>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FA8A025950>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FA8A025950>),
 QuerySQLCheckerTool(description='Use this tool to 

In [9]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "system_message": """
You are an expert SQL assistant helping query a SQLite-based retail database which gives one line answer.

✅ Instructions:
1. If table info is missing, always use `TableInfoAndExamples` first and then check for 
'TableRelationships' for the relationships between tables.

2. Use this response format:
   - Action: <tool-name or Final Answer>
   - Action Input: <input>
3. Execute real SQL queries after table identification.
4. Only show SELECT query results, not intermediate thoughts or tools.
5. When asked about "top discount schemes", use:
SELECT name, discount_percent FROM tbl_scheme WHERE is_active = 1 ORDER BY discount_percent DESC;

6. When asked about MRP of the brands, check in the tbl_product_master
7. When asked about products with have top discounts then use 'TableRelationships' for joining and fetch products.
"""
    }
)

C:\Users\engma\AppData\Local\Temp\ipykernel_27048\3545713887.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [10]:
# Chat input
user_query = input()

 mrp of crocin


In [11]:
response = agent.run(user_query)

C:\Users\engma\AppData\Local\Temp\ipykernel_27048\1665997697.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(user_query)




> Entering new AgentExecutor chain...
I'm happy to help! However, I need more information to provide a meaningful answer. It seems like the question is asking for the molecular recipe of Crocin, which is a chemical compound. Since I'm a large language model, I don't have direct access to databases or chemical compounds. But I can try to help you find the information you're looking for.

Action: sql_db_query
Action Input: SELECT * FROM chemical_database WHERE chemical_name = 'Crocin'
Observation: Error: (sqlite3.OperationalError) no such table: chemical_database
[SQL: SELECT * FROM chemical_database WHERE chemical_name = 'Crocin']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought:Thought: It seems like the database table "chemical_database" doesn't exist. I'll try to list the available tables in the database first.

Action: sql_db_list_tables
Action Input: ''
Observation: order_items, orders, sub_d_order_items, sub_d_orders, tbl_distributor_details, tbl_price_margin

In [14]:
response

"The MRP of Crocin is 252.0.\n\nExplanation: After analyzing the database schema, I found that the table `tbl_product_master` contains the information about products, including their MRP (Maximum Retail Price). I used the SQL query `SELECT SKU_ID, MRP FROM tbl_product_master WHERE LOWER(Brand_Name) LIKE LOWER('%crocin%')` to search for the MRP of Crocin. The result shows that the MRP of Crocin is 252.0."

In [15]:
sql_match = re.search(r"(SELECT\s.+?;)", response, re.IGNORECASE | re.DOTALL)
print(sql_match)

None


In [ ]:
try:
    sql_match = re.search(r"(SELECT\s.+?;)", response, re.IGNORECASE | re.DOTALL)
    if sql_match:
        query = sql_match.group(1).strip()
        result = db.run(query)

        if isinstance(result, list):
            df = pd.DataFrame(result)
            print("Executed SQL Query:")
            print(query)

            # Clean column names for display
            df.columns = [col.replace("_", " ").title() for col in df.columns]

            # Round numeric columns (like discount)
            for col in df.select_dtypes(include=['float']):
                df[col] = df[col].round(2)

            # Format currency columns if MRP exists
            if "Mrp" in df.columns:
                df["Mrp"] = df["Mrp"].apply(lambda x: f"₹{x:.2f}")

            # Display DataFrame
            display(df)

        else:
            print("Query result:")
            print(result)
    else:
        print("No SQL query found in response.")
        print(response)
except Exception as e:
    print("Error:", e)